In [22]:
import pandas as pd

# Load your Compustat output
comp = pd.read_csv("/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/compustat/fundamentals_compustat.csv")

# Clean and standardize tickers
comp['tic'] = comp['tic'].astype(str).str.upper().str.strip()

# Filter to tickers that look valid (letters/numbers only)
comp = comp[comp['tic'].str.match(r'^[A-Z0-9\.-]+$')]

# Drop duplicates and save to CSV
tickers = comp[['tic']].drop_duplicates().rename(columns={'tic': 'ticker'})
tickers.to_csv("/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/compustat_ticker_list.csv", index=False)

print(f"✅ Exported {len(tickers)} tickers to compustat_ticker_list.csv")
tickers.head()

✅ Exported 1735 tickers to compustat_ticker_list.csv


,ticker
0,AIR
1,AAL
2,CECO
3,PNW
4,PRG


In [1]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

# Load tickers
tickers = pd.read_csv(
    "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/compustat_ticker_list.csv"
)['ticker'].tolist()

# Fetch 13 months to be safe
data = yf.download(
    tickers,
    period="400d",
    interval="1d",
    auto_adjust=False,  # price-only, not dividend-adjusted
    group_by='ticker',
    threads=True
)

# Compute 1Y returns relative to exactly 1Y ago
today = pd.Timestamp.today().normalize()
year_ago = today - pd.Timedelta(days=365)

rows = []
for t in tickers:
    try:
        d = data[t][['Close']].dropna().reset_index()
        d['ticker'] = t

        # Align to nearest trading days
        p_now = d.iloc[-1]['Close']
        d_year_ago = d[d['Date'] >= year_ago].iloc[0]['Close']  # first after year_ago
        ret_12m = (p_now / d_year_ago - 1) * 100

        rows.append({'ticker': t, 'ret_12m_pct': ret_12m})
    except Exception:
        continue

returns = pd.DataFrame(rows)
returns.to_csv(
    "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/ret_12m_priceonly.csv",
    index=False
)
print(f"✅ Saved {len(returns)} 1Y price returns (price-only, Yahoo comparable).")


[*********************100%***********************]  1735 of 1735 completed

3 Failed downloads:
['BRK.B', 'X']: YFPricesMissingError('possibly delisted; no price data found  (period=400d) (Yahoo error = "No data found, symbol may be delisted")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=400d)')


✅ Saved 1732 1Y price returns (price-only, Yahoo comparable).


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm

# === SETTINGS ===
input_path = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/compustat_ticker_list.csv"
output_prices = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/daily_prices_yfinance.csv"
output_fundamentals = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/yf_fundamentals.csv"

# === LOAD TICKERS ===
tickers = pd.read_csv(input_path)['ticker'].dropna().unique().tolist()
print(f"Loaded {len(tickers)} tickers")

# === 1) Fetch DAILY adjusted prices ===
data = yf.download(
    tickers,
    period="365d",
    interval="1d",
    auto_adjust=True,
    group_by='ticker',
    threads=True,
    progress=True
)

rows = []
for t in tqdm(tickers, desc="Processing prices"):
    try:
        d = data[t][['Close']].dropna().reset_index()
        d['ticker'] = t
        rows.append(d)
    except Exception:
        continue

daily_prices = pd.concat(rows)
daily_prices.rename(columns={'Date': 'date', 'Close': 'adj_close'}, inplace=True)
daily_prices.to_csv(output_prices, index=False)
print(f"✅ Saved {len(daily_prices):,} daily price observations to {output_prices}")

# === 2) Fetch FUNDAMENTALS (EPS + Dividends) ===
fundamentals = []
for t in tqdm(tickers, desc="Fetching fundamentals"):
    try:
        tk = yf.Ticker(t)

        # Dividend data (cash per share)
        div = tk.dividends
        div_last_yr = div[div.index >= (div.index.max() - pd.Timedelta(days=365))] if len(div) > 0 else pd.Series([])
        div_cash_365d = div_last_yr.sum() if not div_last_yr.empty else np.nan

        # Market info (includes trailing EPS and current price)
        info = tk.info
        eps = info.get("trailingEps", np.nan)
        price = info.get("currentPrice", np.nan)
        marketcap = info.get("marketCap", np.nan)

        # Dividend yield: annual dividend / current price
        div_yield = np.nan
        if price and not np.isnan(price) and div_cash_365d and not np.isnan(div_cash_365d):
            div_yield = div_cash_365d / price

        fundamentals.append({
            "ticker": t,
            "eps_trailing": eps,
            "div_cash_365d": div_cash_365d,
            "div_yield": round(div_yield * 100, 2) if not np.isnan(div_yield) else np.nan,
            "marketcap": marketcap,
            "price": price
        })
    except Exception as e:
        fundamentals.append({
            "ticker": t,
            "eps_trailing": np.nan,
            "div_cash_365d": np.nan,
            "div_yield": np.nan,
            "marketcap": np.nan,
            "price": np.nan
        })

fund_df = pd.DataFrame(fundamentals)
fund_df.to_csv(output_fundamentals, index=False)
print(f"✅ Saved fundamentals for {len(fund_df)} tickers to {output_fundamentals}")

# === 3) Merge daily + fundamentals summary ===
print(fund_df.head(10))



Loaded 1735 tickers


[*********************100%***********************]  1735 of 1735 completed

4 Failed downloads:
['BRK.B', 'X']: YFPricesMissingError('possibly delisted; no price data found  (period=365d) (Yahoo error = "No data found, symbol may be delisted")')
['AEO']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=365d)')
Processing prices: 100%|██████████| 1735/1735 [00:01<00:00, 1073.66it/s]


✅ Saved 629,628 daily price observations to /Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/daily_prices_yfinance.csv


Fetching fundamentals:   0%|          | 6/1735 [00:06<30:00,  1.04s/it]/var/folders/s_/ps4jvmnj60vcm3c12vhqhcfm0000gn/T/ipykernel_10293/4275571763.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  div_last_yr = div[div.index >= (div.index.max() - pd.Timedelta(days=365))] if len(div) > 0 else pd.Series([])
Fetching fundamentals:   2%|▏         | 41/1735 [00:36<25:46,  1.10it/s]$BRK.B: possibly delisted; no timezone found
/Users/paulgrass/anaconda3/lib/python3.11/site-packages/yfinance/scrapers/history.py:506: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
/var/folders/s_/ps4jvmnj60vcm3c12vhqhcfm0000gn/T/ipykernel_10293/4275571763.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future v

✅ Saved fundamentals for 1735 tickers to /Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/yf_fundamentals.csv
  ticker  eps_trailing  div_cash_365d  div_yield     marketcap   price
0    AIR          0.80          0.375       0.44  3.345404e+09   84.56
1    AAL          0.87          0.500       3.93  8.389699e+09   12.71
2   CECO          1.43          0.357       0.74  1.713414e+09   48.50
3    PNW          4.86          4.460       4.92  1.082966e+10   90.68
4    PRG          3.94          0.630       2.00  1.245291e+09   31.49
5    ABT          7.96          2.910       2.30  2.203595e+11  126.61
6    AMD          1.68            NaN        NaN  4.187099e+11  258.01
7    ALK          1.21          1.775       4.13  4.954999e+09   43.00
8   MATX         14.69          1.380       1.41  3.106105e+09   97.71
9    ALX          7.27         22.500       9.74  1.179529e+09  230.95


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# --------- SETTINGS ---------
BASE = Path("/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection")

# Inputs
PATH_COMPUSTAT   = BASE / "compustat/fundamentals_compustat.csv"
PATH_DAILY       = BASE / "daily_prices_yfinance.csv"
PATH_YF_FUNDS    = BASE / "yf_fundamentals.csv"

# Output
PATH_MASTER      = BASE / "merged_master.csv"

# Optional: save quick per-sector summaries
SAVE_SECTOR_SPLITS = True
# --------------------------------


# 1) Load Compustat fundamentals and build P/B
comp = pd.read_csv(PATH_COMPUSTAT)

# Clean tickers
comp['tic'] = comp['tic'].astype(str).str.upper().str.strip()

# Preferred stock: take first non-missing among pstkrv, pstkl, pstk
pstk_series = comp[['pstkrv','pstkl','pstk']].copy()
pstk_val = pstk_series.bfill(axis=1).iloc[:,0].fillna(0)

# Book equity and P/B (MKVALT is in millions USD)
comp['book_equity'] = comp['ceq'].fillna(0) + pstk_val
comp['pb'] = comp['mkvalt'] / comp['book_equity'].replace(0, np.nan)

# Optional pretties
comp['pb'] = comp['pb'].astype(float)
comp['marketcap_musd'] = comp['mkvalt'].astype(float)
comp['marketcap_usd'] = comp['marketcap_musd'] * 1e6
comp['marketcap_bn'] = (comp['marketcap_usd'] / 1e9).round(2)

# Keep lean columns from Compustat
keep_comp_cols = [
    'gvkey','tic','conm','gsector','gind','datadate',
    'marketcap_musd','marketcap_bn','pb','book_equity'
]
comp_small = comp[keep_comp_cols].rename(columns={
    'tic':'ticker',
    'conm':'company',
    'gsector':'gsector_code',
    'gind':'gind_code',
    'datadate':'fund_datadate'
})

# Complete GICS Sector mapping (Compustat GSECTOR codes)
GSECTOR_NAME = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",                 # ✅ your main focus
    25: "Consumer Discretionary",
    30: "Consumer Staples / Defensive",# ✅ your main focus
    35: "Health Care",
    40: "Financials",                  # ❌ exclude (P/B problematic)
    45: "Information Technology",      # ✅ your main focus
    50: "Communication Services",
    55: "Utilities",                   # ❌ exclude (regulated assets)
    60: "Real Estate"                  # ❌ exclude (book ≠ economic value)
}
comp_small['gsector_name'] = comp_small['gsector_code'].map(GSECTOR_NAME)


# 2) Load daily prices, compute 12m total return (already div-adjusted)
prices = pd.read_csv(PATH_DAILY, parse_dates=['date'])
prices['ticker'] = prices['ticker'].astype(str).str.upper().str.strip()

# Compute total return over the available ~365 days panel per ticker:
# ret_12m = last adj_close / first adj_close - 1
agg = (prices.sort_values(['ticker','date'])
              .groupby('ticker')
              .agg(first_price=('adj_close','first'),
                   last_price=('adj_close','last'),
                   start_date=('date','first'),
                   end_date=('date','last'))
              .reset_index())
agg['ret_12m'] = agg['last_price'] / agg['first_price'] - 1.0
agg['ret_12m_pct'] = (agg['ret_12m'] * 100).round(2)


# 3) Load Yahoo fundamentals (EPS, dividends, yield)
yf = pd.read_csv(PATH_YF_FUNDS)
yf['ticker'] = yf['ticker'].astype(str).str.upper().str.strip()

# Make sure expected columns exist (fill if missing)
for col in ['eps_trailing','div_cash_365d','div_yield','marketcap','price']:
    if col not in yf.columns:
        yf[col] = np.nan

# 4) Merge: Compustat ← returns ← Yahoo fundamentals
merged = (comp_small
          .merge(agg[['ticker','ret_12m','ret_12m_pct','start_date','end_date']],
                 on='ticker', how='left')
          .merge(yf[['ticker','eps_trailing','div_cash_365d','div_yield','marketcap','price']],
                 on='ticker', how='left'))

# 5) Final tidy columns & sorting
cols_order = [
    'ticker','company','gvkey','gsector_code','gsector_name','gind_code','fund_datadate',
    'marketcap_musd','marketcap_bn','book_equity','pb',
    'start_date','end_date','ret_12m','ret_12m_pct',
    'price','marketcap','eps_trailing','div_cash_365d','div_yield'
]
# Keep only those that exist
cols_order = [c for c in cols_order if c in merged.columns]
merged = merged[cols_order].sort_values(['gsector_code','company','ticker'])

# 6) Save master CSV
merged.to_csv(PATH_MASTER, index=False)
print(f"✅ Saved merged master to: {PATH_MASTER}")
print(f"Rows: {len(merged):,}")
print(merged.head(10).to_string(index=False))

# 7) (Optional) Save per-sector splits for convenience
if SAVE_SECTOR_SPLITS:
    for code, name in GSECTOR_NAME.items():
        sub = merged[merged['gsector_code']==code]
        if not sub.empty:
            outp = BASE / f"merged_master_{name.replace(' ','')}.csv"
            sub.to_csv(outp, index=False)
            print(f"↳ Saved {name}: {len(sub):,} rows to {outp}")


✅ Saved merged master to: /Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master.csv
Rows: 1,735
ticker                      company  gvkey  gsector_code gsector_name  gind_code fund_datadate  marketcap_musd  marketcap_bn  book_equity       pb start_date   end_date   ret_12m  ret_12m_pct  price    marketcap  eps_trailing  div_cash_365d  div_yield
  ARLP ALLIANCE RESOURCE PTNRS  -LP 122915            10       Energy     101020    2024-12-31       3366.7500          3.37     1832.747 1.836997 2024-05-15 2025-10-28  0.214684        21.47  24.31 3.122085e+09          1.89          3.400      13.99
    AM        ANTERO MIDSTREAM CORP  31109            10       Energy     101020    2024-12-31       7234.4780          7.23     2115.171 3.420280 2024-05-15 2025-10-28  0.304919        30.49  17.61 8.424818e+09          0.95          1.125       6.39
    AR        ANTERO RESOURCES CORP  18465            10       Energy     1010

OSError: Cannot save file into a non-existent directory: '/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master_ConsumerStaples'

In [4]:
import pandas as pd
import numpy as np

# ========= PATHS =========
PATH_MASTER = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master.csv"
PATH_OUT    = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master_with_pb.csv"
# =========================

# 1️⃣ Load data
merged = pd.read_csv(PATH_MASTER)
print(f"Loaded {len(merged):,} rows from merged_master.csv")

# 2️⃣ Clean variables
merged["book_equity"] = pd.to_numeric(merged["book_equity"], errors="coerce")
merged["marketcap_musd"] = pd.to_numeric(merged["marketcap_musd"], errors="coerce")
merged["marketcap"] = pd.to_numeric(merged["marketcap"], errors="coerce")

# Replace zeros and negatives in book_equity with NaN
merged.loc[merged["book_equity"] <= 0, "book_equity"] = np.nan

# 3️⃣ Compute P/B (fiscal-year-end and current)
# marketcap_musd is in millions USD → divide by book_equity (same units)
merged["pb_fye"] = merged["marketcap_musd"] / merged["book_equity"]

# marketcap from Yahoo is in USD → divide by (book_equity * 1e6)
merged["pb_current"] = merged["marketcap"] / (merged["book_equity"] * 1e6)

# Drop or flag nonsensical (<= 0 or infinite) P/Bs
merged.loc[~np.isfinite(merged["pb_current"]), "pb_current"] = np.nan
merged.loc[merged["pb_current"] <= 0, "pb_current"] = np.nan

merged.loc[~np.isfinite(merged["pb_fye"]), "pb_fye"] = np.nan
merged.loc[merged["pb_fye"] <= 0, "pb_fye"] = np.nan

# 4️⃣ Round for readability
merged["pb_fye"] = merged["pb_fye"].round(2)
merged["pb_current"] = merged["pb_current"].round(2)

# 5️⃣ Compute sector-relative percentile of pb_current (only for valid values)
merged["pb_current_pctile"] = (
    merged.groupby("gsector_code")["pb_current"]
    .rank(pct=True, method="average")
)

# 6️⃣ Create categorical valuation label (e.g. low/high)
def label_val(p):
    if pd.isna(p):
        return np.nan
    elif p <= 0.3:
        return "Low valuation"
    elif p >= 0.7:
        return "High valuation"
    else:
        return "Mid valuation"

merged["valuation_label"] = merged["pb_current_pctile"].apply(label_val)

# 7️⃣ Save updated file
merged.to_csv(PATH_OUT, index=False)
print(f"✅ Saved file with CLEAN positive P/B measures to: {PATH_OUT}")

# Quick preview
cols = [
    "ticker",
    "company",
    "gsector_name",
    "pb_fye",
    "pb_current",
    "pb_current_pctile",
    "valuation_label",
]
print(merged[cols].head(10).to_string(index=False))

# Optional diagnostic
n_invalid = merged["pb_current"].isna().sum()
print(f"ℹ️ {n_invalid:,} rows have invalid or nonpositive P/B (set to NaN).")


Loaded 1,735 rows from merged_master.csv
✅ Saved file with CLEAN positive P/B measures to: /Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master_with_pb.csv
ticker                      company gsector_name  pb_fye  pb_current  pb_current_pctile valuation_label
  ARLP ALLIANCE RESOURCE PTNRS  -LP       Energy    1.84        1.70           0.459770   Mid valuation
    AM        ANTERO MIDSTREAM CORP       Energy    3.42        3.98           0.839080  High valuation
    AR        ANTERO RESOURCES CORP       Energy    1.55        1.40           0.379310   Mid valuation
   APA                     APA CORP       Energy    1.60        1.54           0.402299   Mid valuation
  AROC                 ARCHROCK INC       Energy    3.29        3.41           0.816092  High valuation
  AESI   ATLAS ENERGY SOLUTIONS INC       Energy    2.36        1.66           0.431034   Mid valuation
   BKR              BAKER HUGHES CO       Ene

In [5]:
import pandas as pd
import numpy as np

# === 1. Load the merged dataset ===
merged_path = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master_with_pb.csv"
df = pd.read_csv(merged_path)

# === 2. Merge in the corrected price-only returns ===
returns_path = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/ret_12m_priceonly.csv"
returns = pd.read_csv(returns_path)

# Ensure proper merge key
returns.rename(columns={"ret_12m_pct": "ret_12m_priceonly_pct"}, inplace=True)
df = df.merge(returns, on="ticker", how="left")

# === 3. Compute updated return brackets ===
# Drop missing
df = df.dropna(subset=["ret_12m_priceonly_pct"])

# Compute quantile cutoffs
q1 = df["ret_12m_priceonly_pct"].quantile(1/3)
q2 = df["ret_12m_priceonly_pct"].quantile(2/3)

def classify_return(x):
    if x <= q1:
        return "LOW"
    elif x <= q2:
        return "MED"
    else:
        return "HIGH"

df["return_band"] = df["ret_12m_priceonly_pct"].apply(classify_return)

# === 4. Save updated subsets ===
base_path = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection"

df_low = df[df["return_band"] == "LOW"]
df_med = df[df["return_band"] == "MED"]
df_high = df[df["return_band"] == "HIGH"]

df_low.to_csv(f"{base_path}/StockSelection_Low.csv", index=False)
df_med.to_csv(f"{base_path}/StockSelection_Medium.csv", index=False)
df_high.to_csv(f"{base_path}/StockSelection_High.csv", index=False)

# === 5. Save the fully updated master ===
df.to_csv(f"{base_path}/merged_master_updated.csv", index=False)

print(f"✅ Updated brackets based on price-only returns.")
print(f"Saved {len(df_low)} LOW, {len(df_med)} MED, {len(df_high)} HIGH.")


✅ Updated brackets based on price-only returns.
Saved 577 LOW, 577 MED, 578 HIGH.


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# ========= PATHS =========
BASE = Path("/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection")

PATH_MASTER_IN   = BASE / "merged_master_updated.csv"    # <-- already has pb_current_pctile, valuation_label, return_band, etc.
OUT_DIR = BASE / "preselection"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Prescreen outputs (kept for visibility)
PATH_OUT_ALL   = OUT_DIR / "candidate_subset_all.csv"
PATH_OUT_PRE   = OUT_DIR / "candidate_subset_prescreened.csv"
PATH_OUT_LOW   = OUT_DIR / "candidate_subset_LOW.csv"
PATH_OUT_MED   = OUT_DIR / "candidate_subset_MED.csv"
PATH_OUT_HIGH  = OUT_DIR / "candidate_subset_HIGH.csv"

# Candidate PAIRS outputs
PATH_CAND_LOW_TECH        = OUT_DIR / "candidate_pairs_LOW_TECH.csv"
PATH_CAND_MED_IND         = OUT_DIR / "candidate_pairs_MED_INDUSTRIALS.csv"
PATH_CAND_HIGH_TECH       = OUT_DIR / "candidate_pairs_HIGH_TECH.csv"
PATH_CAND_ANCHORS_FAMILIAR= OUT_DIR / "candidate_pairs_ANCHORS_FAMILIAR.csv"

# ------- SETTINGS -------
BANDS = {
    "LOW":  (-25.0, -10.0),
    "MED":  (  5.0,  20.0),
    "HIGH": ( 40.0,  70.0),
}
SECTOR_TECH        = "Information Technology"
SECTOR_INDUSTRIALS = "Industrials"

TOP_N_PAIRS_PER_GROUP = 70         # number of pairs per core group
TOP_N_ANCHOR_PAIRS    = 350        # number of familiar anchor pairs (set None to keep all)

# Familiar anchors constraint
ANCHOR_MIN_MKT_CAP = 10e10          # > $50bn
EXCLUDE_SECTORS_FOR_ANCHORS = {"Financials", "Real Estate"}

# ---------- Helpers ----------
def save_csv(df: pd.DataFrame, path: Path, label: str = ""):
    df.to_csv(path, index=False)
    if label:
        print(f"✅ Saved {label}: {path.name} (rows={len(df)})")

def label_band(x_pct: float) -> str | None:
    if pd.isna(x_pct):
        return None
    for name, (lo, hi) in BANDS.items():
        if lo <= x_pct <= hi:
            return name
    return None

def generate_pair_candidates(df_band_scope: pd.DataFrame,
                             ret_col="ret_12m_priceonly_pct",
                             val_pct_col="pb_current_pctile",
                             top_n=10,
                             max_ret_diff_primary=3.0,
                             max_ret_diff_fallback=5.0,
                             low_thr=0.30,
                             high_thr=0.70,
                             # scoring weights
                              w_val=3.0,                 # valuation contrast weight
                             w_ret=2.0,                 # return closeness penalty weight
                             # anchor-specific extras
                             prefer_band: str | None = None,
                             prefer_boost=(0.20, 0.07), # (both-in-band bonus, one-in-band bonus)
                             marketcap_weight=0.0):     # >0 to reward large caps (anchors)
    """
    Build MANY ranked pair candidates within df_band_scope:

      1) strict valuation split (<=low_thr vs >=high_thr), |Δret| <= 3pp
      2) strict + 5pp
      3) loose halves (<0.5 vs >=0.5) + 3pp
      4) loose halves + 5pp

    Score = w_val*valuation_contrast - w_ret*(ret_diff/max_allowed)
            + marketcap_weight * pair_mcap_pref
            + band_bonus (if prefer_band is set)

    pair_mcap_pref = average of each leg’s market-cap percentile within df_band_scope.
    """
    req = {"ticker", ret_col, val_pct_col}
    if not req.issubset(df_band_scope.columns):
        return pd.DataFrame(columns=[])

    df_ = df_band_scope.dropna(subset=[ret_col, val_pct_col, "ticker"]).copy()
    if df_.empty:
        return df_

    # Precompute single-name market-cap percentile (ascending: larger cap => higher pct)
    if "marketcap" in df_.columns:
        mcap_rank = (
            df_[["ticker", "marketcap"]]
            .assign(mcap_pct=lambda x: x["marketcap"].rank(pct=True))
        )
    else:
        mcap_rank = None

    # Prepare strict / loose cohorts
    lo_strict  = df_[df_[val_pct_col] <= low_thr].copy()
    hi_strict  = df_[df_[val_pct_col] >= high_thr].copy()
    lo_loose   = df_[df_[val_pct_col] <  0.5].copy()
    hi_loose   = df_[df_[val_pct_col] >= 0.5].copy()

    def _score(lo_df, hi_df, max_ret_diff):
        if lo_df.empty or hi_df.empty:
            return pd.DataFrame()

        lo_df = lo_df.assign(__key=1)
        hi_df = hi_df.assign(__key=1)
        m = lo_df.merge(hi_df, on="__key", suffixes=("_lo","_hi")).drop(columns="__key")

        # Filter self-pairs and enforce return closeness
        m = m[m["ticker_lo"] != m["ticker_hi"]]
        m["ret_diff_pp"] = (m[f"{ret_col}_lo"] - m[f"{ret_col}_hi"]).abs()
        m = m[m["ret_diff_pp"] <= max_ret_diff]
        if m.empty:
            return m

        # P/B percentile contrast
        m["val_contrast"] = (m[f"{val_pct_col}_hi"] - m[f"{val_pct_col}_lo"]).abs()

        # Market-cap preference (only if requested)
        if marketcap_weight > 0.0 and mcap_rank is not None:
            m = m.merge(mcap_rank.rename(columns={"ticker":"ticker_lo", "mcap_pct":"mcap_pct_lo"}),
                        on="ticker_lo", how="left")
            m = m.merge(mcap_rank.rename(columns={"ticker":"ticker_hi", "mcap_pct":"mcap_pct_hi"}),
                        on="ticker_hi", how="left")
            m["pair_mcap_pref"] = (m["mcap_pct_lo"].fillna(0) + m["mcap_pct_hi"].fillna(0)) / 2.0
        else:
            m["pair_mcap_pref"] = 0.0

        # Composite score
        m["score"] = (
            w_val * m["val_contrast"]
            - w_ret * (m["ret_diff_pp"] / max_ret_diff)
            + marketcap_weight * m["pair_mcap_pref"]
        )

        # Optional band preference (e.g., prefer MED for anchors)
        if prefer_band is not None and "return_band_lo" in m.columns and "return_band_hi" in m.columns:
            both = (m["return_band_lo"] == prefer_band) & (m["return_band_hi"] == prefer_band)
            one  = ((m["return_band_lo"] == prefer_band) ^ (m["return_band_hi"] == prefer_band))
            bonus = np.where(both, prefer_boost[0], np.where(one, prefer_boost[1], 0.0))
            m["score"] = m["score"] + bonus

        # Deduplicate unordered pairs, keep best scoring orientation
        m["pair_key"] = m.apply(lambda r: tuple(sorted([r["ticker_lo"], r["ticker_hi"]])), axis=1)
        m = m.sort_values("score", ascending=False).drop_duplicates("pair_key", keep="first")
        return m

    buckets = [
        _score(lo_strict, hi_strict, max_ret_diff_primary),
        _score(lo_strict, hi_strict, max_ret_diff_fallback),
        _score(lo_loose,  hi_loose,  max_ret_diff_primary),
        _score(lo_loose,  hi_loose,  max_ret_diff_fallback),
    ]

    out = pd.concat([b for b in buckets if b is not None and not b.empty], ignore_index=True)
    if out.empty:
        return out

    out = out.sort_values("score", ascending=False)

    keep_cols = [
        # left leg
        "ticker_lo","company_lo","gsector_name_lo","return_band_lo",
        f"{ret_col}_lo", f"{val_pct_col}_lo",
        "marketcap_lo","div_yield_lo","eps_tra_lo",
        # right leg
        "ticker_hi","company_hi","gsector_name_hi","return_band_hi",
        f"{ret_col}_hi", f"{val_pct_col}_hi",
        "marketcap_hi","div_yield_hi","eps_tra_hi",
        # diagnostics
        "ret_diff_pp","val_contrast","pair_mcap_pref","score"
    ]
    keep_cols = [c for c in keep_cols if c in out.columns]
    return out[keep_cols].head(top_n) if (top_n is not None) else out[keep_cols]

# ===================== 1) LOAD (NO RECOMPUTATION OF P/B) =====================
df = pd.read_csv(PATH_MASTER_IN)
print(f"Loaded {len(df):,} rows from {PATH_MASTER_IN.name}")

# Ensure numerics for features we use
for c in ["ret_12m_pct","ret_12m_priceonly_pct","pb_current","pb_current_pctile","div_yield","eps_trailing","eps_tra","marketcap"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
# Back-compat for EPS column
if "eps_trailing" in df.columns and "eps_tra" not in df.columns:
    df["eps_tra"] = df["eps_trailing"]

# (Re)compute return bands from PRICE-ONLY returns to be safe
if "ret_12m_priceonly_pct" in df.columns:
    df["return_band"] = df["ret_12m_priceonly_pct"].apply(label_band)

# Slim view for prescreen/debug
keep_cols = [
    "ticker","company","gsector_name","gsector_code","return_band",
    "ret_12m_pct","ret_12m_priceonly_pct",
    "eps_tra","div_yield",
    "pb_current","pb_current_pctile","valuation_label",
    "marketcap"
]
keep_cols = [c for c in keep_cols if c in df.columns]
subset_all = df[keep_cols].copy()
subset_all["marketcap_bn"] = (subset_all["marketcap"] / 1e9).round(2) if "marketcap" in subset_all else np.nan

save_csv(subset_all, PATH_OUT_ALL, "full subset (using updated master)")

# Prescreen by bands (using price-only)
subset_all["return_band"] = subset_all["ret_12m_priceonly_pct"].apply(label_band) if "ret_12m_priceonly_pct" in subset_all else np.nan
subset_pre = subset_all.dropna(subset=["return_band"]).copy()
save_csv(subset_pre, PATH_OUT_PRE, "prescreened subset (any band)")

for band, path in [("LOW", PATH_OUT_LOW), ("MED", PATH_OUT_MED), ("HIGH", PATH_OUT_HIGH)]:
    sub = subset_pre[subset_pre["return_band"] == band].copy()
    save_csv(sub, path, f"band prescreen {band}")
    print(f"↳ {band}: {len(sub)} rows")

# ===================== 2) MANY PAIRS PER CORE GROUP (same-sector) =====================
# LOW / Tech
low_tech = subset_pre.query("return_band == 'LOW' and gsector_name == @SECTOR_TECH").copy()
cand_low_tech = generate_pair_candidates(
    low_tech,
    ret_col="ret_12m_priceonly_pct",
    val_pct_col="pb_current_pctile",
    top_n=TOP_N_PAIRS_PER_GROUP
)
save_csv(cand_low_tech, PATH_CAND_LOW_TECH, "LOW/TECH candidate pairs")

# MED / Industrials
med_ind = subset_pre.query("return_band == 'MED' and gsector_name == @SECTOR_INDUSTRIALS").copy()
cand_med_ind = generate_pair_candidates(
    med_ind,
    ret_col="ret_12m_priceonly_pct",
    val_pct_col="pb_current_pctile",
    top_n=TOP_N_PAIRS_PER_GROUP
)
save_csv(cand_med_ind, PATH_CAND_MED_IND, "MED/INDUSTRIALS candidate pairs")

# HIGH / Tech
high_tech = subset_pre.query("return_band == 'HIGH' and gsector_name == @SECTOR_TECH").copy()
cand_high_tech = generate_pair_candidates(
    high_tech,
    ret_col="ret_12m_priceonly_pct",
    val_pct_col="pb_current_pctile",
    top_n=TOP_N_PAIRS_PER_GROUP
)
save_csv(cand_high_tech, PATH_CAND_HIGH_TECH, "HIGH/TECH candidate pairs")

# ===================== 3) FAMILIAR ANCHOR PAIRS (large-cap, cross-sector allowed, prefer MED) =====================
anchors_pool = subset_all.copy()
anchors_pool = anchors_pool[
    (anchors_pool["marketcap"].fillna(0) >= ANCHOR_MIN_MKT_CAP) &
    (~anchors_pool["gsector_name"].isin(EXCLUDE_SECTORS_FOR_ANCHORS))
].copy()

cand_anchors = generate_pair_candidates(
    anchors_pool,
    ret_col="ret_12m_priceonly_pct",
    val_pct_col="pb_current_pctile",
    top_n=TOP_N_ANCHOR_PAIRS,
    prefer_band="MED",            # prefer MED but don't require it
    prefer_boost=(0.20, 0.07),
    marketcap_weight=1.0          # reward larger caps in the pair
)
save_csv(cand_anchors, PATH_CAND_ANCHORS_FAMILIAR, "FAMILIAR large-cap anchor pairs")

# ===================== 4) PREVIEW =====================
def prev(df, name):
    if df is None or df.empty:
        print(f"⚠️ No pairs for {name}. Consider widening return diff or loosening valuation split.")
    else:
        cols = [c for c in ["ticker_lo","ticker_hi","return_band_lo","return_band_hi","ret_diff_pp","val_contrast","pair_mcap_pref","score"] if c in df.columns]
        print(f"\nTop pairs for {name}:")
        print(df[cols].head(10).to_string(index=False))

prev(cand_low_tech,        "LOW/TECH")
prev(cand_med_ind,         "MED/INDUSTRIALS")
prev(cand_high_tech,       "HIGH/TECH")
prev(cand_anchors,         "FAMILIAR ANCHORS")


Loaded 1,732 rows from merged_master_updated.csv
✅ Saved full subset (using updated master): candidate_subset_all.csv (rows=1732)
✅ Saved prescreened subset (any band): candidate_subset_prescreened.csv (rows=718)
✅ Saved band prescreen LOW: candidate_subset_LOW.csv (rows=324)
↳ LOW: 324 rows
✅ Saved band prescreen MED: candidate_subset_MED.csv (rows=271)
↳ MED: 271 rows
✅ Saved band prescreen HIGH: candidate_subset_HIGH.csv (rows=123)
↳ HIGH: 123 rows
✅ Saved LOW/TECH candidate pairs: candidate_pairs_LOW_TECH.csv (rows=70)
✅ Saved MED/INDUSTRIALS candidate pairs: candidate_pairs_MED_INDUSTRIALS.csv (rows=70)
✅ Saved HIGH/TECH candidate pairs: candidate_pairs_HIGH_TECH.csv (rows=31)
✅ Saved FAMILIAR large-cap anchor pairs: candidate_pairs_ANCHORS_FAMILIAR.csv (rows=94)

Top pairs for LOW/TECH:
ticker_lo ticker_hi return_band_lo return_band_hi  ret_diff_pp  val_contrast  pair_mcap_pref    score
     UCTT      TEAM            LOW            LOW     0.545797      0.823256             0.0 2

In [34]:
import pandas as pd

# ========= PATH =========
PATH_MASTER = "/Users/paulgrass/Library/Mobile Documents/com~apple~CloudDocs/Documents/Model Spillovers/Data/Stock Selection/merged_master_updated.csv"
# ========================

# 1️⃣ Load data
df = pd.read_csv(PATH_MASTER)

# 2️⃣ Ensure numeric consistency
df["marketcap"] = pd.to_numeric(df["marketcap"], errors="coerce")
df["pb_current_pctile"] = pd.to_numeric(df["pb_current_pctile"], errors="coerce")

# 3️⃣ Define "very large cap" threshold (>= $100B)
VERY_LARGE_CAP = 1e11  # USD

# 4️⃣ Filter: large cap and mid-range valuation (0.40–0.60)
mask = (
    (df["marketcap"] >= VERY_LARGE_CAP)
    & (df["pb_current_pctile"].between(0.33, 0.66))
)

large_midpb = df.loc[mask, [
    "ticker",
    "company",
    "gsector_name",
    "pb_current",
    "pb_current_pctile",
    "marketcap",
    "ret_12m_priceonly_pct"
]].sort_values("pb_current")

# 5️⃣ Format market cap in billions for readability
large_midpb["marketcap_bn"] = (large_midpb["marketcap"] / 1e9).round(1)
large_midpb = large_midpb.drop(columns="marketcap")

# 6️⃣ Print results
print(f"✅ Found {len(large_midpb)} very large-cap companies with 0.40 ≤ P/B ≤ 0.60:")
print(large_midpb.to_string(index=False))


✅ Found 21 very large-cap companies with 0.40 ≤ P/B ≤ 0.60:
ticker                      company           gsector_name  pb_current  pb_current_pctile  ret_12m_priceonly_pct  marketcap_bn
   BAC         BANK OF AMERICA CORP             Financials        1.29           0.418301              24.940137         382.2
   WFC             WELLS FARGO & CO             Financials        1.50           0.513072              30.084354         269.3
   COP               CONOCOPHILLIPS                 Energy        1.66           0.425287             -17.573100         107.7
     T                     AT&T INC Communication Services        1.79           0.354839              20.290861         186.4
   XOM             EXXON MOBIL CORP                 Energy        1.82           0.534483              -6.387407         478.8
    GS      GOLDMAN SACHS GROUP INC             Financials        1.93           0.602941              47.603321         235.3
   CVX                 CHEVRON CORP                